### Questions set out to answer - Jayden LI
    Topic: Operators
    Backlog: 010, 014
* Owners (operators) of wind turbines in the USA
* Wind energy generation per owner
* Which other types of power the wind owners already own
* Identify potential companies to approach to fund

    ## Data Preparation

In [3]:
-- SQL Code to extract information of the wind power plants from 'EIA923_Schedules_2_3_4_5_M_12_2020_Final_Revision':
-- Excluded 162 rows of ‘State-Fuel Level Increment’(id:99999) with plant_id and operator_id 99999, as it is defined to ‘represents cumulative emissions from facilities that are too small to trigger EIA's specific reporting’
-- Excluded 1 row of Null values.

SELECT plant_id
       ,combined_heat_and_power_plant
       ,plant_name
       ,operator_name
       ,operator_id
       ,plant_state
       ,reported_prime_mover
       ,reported_fuel_type_code
       ,total_fuel_consumption_mm_btu
       ,net_generation_megawatthours
       ,year
FROM public.eia923_operators
WHERE plant_id != '99999' AND plant_state != 'NULL'

-- the file is then exported to 'Operator Data Extracted_V1.08.10.22'

SyntaxError: invalid character '‘' (U+2018) (2179632901.py, line 3)

In [44]:
# In this step, us python to create two new columns in the csv file
## 1. heat_rate by dividing 'total_fuel_consumption_mm_btu' by 'net_generation_megawatthours'
## 2. plan_efficiency by dividing 'heat_rate' with the constant 3412 Btu
### Reference: https://www.eia.gov/tools/faqs/faq.php?id=107&t=3

# First import libary statement

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint as pp
import seaborn as sns

In [45]:
# Check columns in the csv file:
# No Null or missing data

operatorDf = pd.read_csv('./Operator Data Extracted_V1_08.10.22.csv')
operatorDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14952 entries, 0 to 14951
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   plant_id                       14952 non-null  int64 
 1   combined_heat_and_power_plant  14952 non-null  object
 2   plant_name                     14952 non-null  object
 3   operator_name                  14952 non-null  object
 4   operator_id                    14952 non-null  int64 
 5   plant_state                    14952 non-null  object
 6   reported_prime_mover           14952 non-null  object
 7   reported_fuel_type_code        14952 non-null  object
 8   total_fuel_consumption_mm_btu  14952 non-null  int64 
 9   net_generation_megawatthours   14952 non-null  int64 
 10  year                           14952 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 1.3+ MB


In [46]:
# Calculate heat rate of each plant:

operatorDf['heat_rate'] = operatorDf['total_fuel_consumption_mm_btu']/operatorDf['net_generation_megawatthours']*1000
operatorDf.head()

,plant_id,combined_heat_and_power_plant,plant_name,operator_name,operator_id,plant_state,reported_prime_mover,reported_fuel_type_code,total_fuel_consumption_mm_btu,net_generation_megawatthours,year,heat_rate
0,1,N,Sand Point,"TDX Sand Point Generating, LLC",63560,AK,WT,WND,3045,347,2020,8775.216138
1,2,N,Bankhead Dam,Alabama Power Co,195,AL,HY,WAT,1220939,139170,2020,8773.004239
2,3,N,Barry,Alabama Power Co,195,AL,CA,NG,1243854,2633517,2020,472.316678
3,3,N,Barry,Alabama Power Co,195,AL,CT,NG,53006762,5084350,2020,10425.474643
4,3,N,Barry,Alabama Power Co,195,AL,ST,BIT,26188716,2625835,2020,9973.481197


In [47]:
# Creating the column of 'plan_efficiency' - results are quite similar

operatorDf['plant_efficiency'] = 3412/operatorDf['heat_rate']
operatorDf.head()

,plant_id,combined_heat_and_power_plant,plant_name,operator_name,operator_id,plant_state,reported_prime_mover,reported_fuel_type_code,total_fuel_consumption_mm_btu,net_generation_megawatthours,year,heat_rate,plant_efficiency
0,1,N,Sand Point,"TDX Sand Point Generating, LLC",63560,AK,WT,WND,3045,347,2020,8775.216138,0.388822
1,2,N,Bankhead Dam,Alabama Power Co,195,AL,HY,WAT,1220939,139170,2020,8773.004239,0.388920
2,3,N,Barry,Alabama Power Co,195,AL,CA,NG,1243854,2633517,2020,472.316678,7.223967
3,3,N,Barry,Alabama Power Co,195,AL,CT,NG,53006762,5084350,2020,10425.474643,0.327275
4,3,N,Barry,Alabama Power Co,195,AL,ST,BIT,26188716,2625835,2020,9973.481197,0.342107


    ## Data Exploration

In [48]:
# Net electricity generation mean value (wind operator):

windOperatorDf = operatorDf[operatorDf['reported_prime_mover'].isin(['WT','WS'])]
windOperatorDf.total_fuel_consumption_mm_btu.mean()

2399854.2599190283

In [49]:
# Net electricity generation median value (wind operator):

windOperatorDf.total_fuel_consumption_mm_btu.median()


1420867.0

In [50]:
# Number of operators and their plants count

windOperatorDf.operator_id.value_counts()

15399    57
12341    31
57170    31
56215    28
2770     28
         ..
56654     1
56653     1
58150     1
56625     1
63993     1
Name: operator_id, Length: 652, dtype: int64

In [51]:
# Number of operator names and their plants count
# Seems like Operator IDs and Names are uniquely matched.

windOperatorDf.operator_name.value_counts()

Avangrid Renewables LLC               57
MidAmerican Energy Co                 31
EDF Renewable Asset Holdings, Inc.    31
RWE Renewables Americas LLC           28
Terra-Gen Operating Co-Wind           28
                                      ..
Carleton College                       1
Northern Colorado Wind LLC             1
Big Blue Wind Farm LLC (MN)            1
Flat Water Wind Farm LLC               1
Milligan 1 Wind, LLC                   1
Name: operator_name, Length: 652, dtype: int64

In [52]:
# Operators of wind turbines and their total electricity generated

windOperatorDf.groupby('operator_name')['net_generation_megawatthours'].agg(['mean', 'median', 'min', 'max']).sort_values(by = 'mean', ascending= False)

,mean,median,min,max
operator_name,,,,
FPL Energy Horse Hollow LLC,2513457.0,2513457.0,2513457,2513457
Capricorn Ridge Wind LLC,2259411.0,2259411.0,2259411,2259411
Hale Wind Energy,2083677.0,2083677.0,2083677,2083677
Stephens Ranch Wind Energy LLC,1358663.0,1358663.0,1358663,1358663
Public Service Co of Colorado,1308727.5,1308727.5,593134,2024321
...,...,...,...,...
"Rock County Energy Center, LLC",0.0,0.0,0,0
Tennessee Valley Authority,0.0,0.0,0,0
Ipswich Wind Independence,0.0,0.0,0,0


In [53]:
# Number of wind operators in each state

windOperatorDf.groupby('plant_state')['operator_id'].count().sort_values(ascending= False)

plant_state
TX    190
MN    128
CA    114
IA    112
OK     57
OR     47
IL     46
KS     41
ND     40
ID     32
NE     31
CO     29
MI     28
NY     27
WY     27
PA     26
WA     23
MA     23
NM     23
OH     21
SD     20
ME     18
IN     18
MT     17
RI     17
MO     12
WI     12
HI      8
WV      7
AK      7
AZ      6
MD      5
NH      5
UT      5
VT      5
NJ      2
TN      2
DE      1
NV      1
CT      1
NC      1
Name: operator_id, dtype: int64

In [54]:
# Operators and their plant types - grouped by operators

operatorDf.groupby('operator_name')['reported_prime_mover'].value_counts()

operator_name                      reported_prime_mover
126 Grove Solar LLC                PV                       1
13 Mile Solar, LLC                 PV                       1
158th Fighter Wing                 PV                       1
180 Raritan Energy Solutions, LLC  PV                       1
2016 ESA Project Company, LLC      FC                      31
                                                           ..
Zion Energy LLC                    GT                       2
Zotos International                WT                       1
Zumbro Garden LLC                  PV                       1
Zumbro Solar LLC                   PV                       1
esVolta LP                         BA                       1
Name: reported_prime_mover, Length: 6274, dtype: int64

In [55]:
# Further analysis will be done in excel and tableau
# Export dataframe back to csv and share with teammembers
## further exploration in Tableau

operatorDf.to_csv('Operator Data Extracted_V1_08.10.22.csv', index= False)